In [115]:
import polars as pl
import os

def import_tennis_matches(base_path="matches_and_ranking", data_type="singles"):
    dataframes = []
    null_values = ['Q', 'NA', 'N/A', '', '-', 'Unknown', 'null', 'W/O']
   
    # Define file prefixes for different match types
    match_types = {
        "singles": ["atp_matches"],
        "doubles": ["atp_matches_doubles"],
        "qualifiers_challengers": ["atp_matches_qual_chall"],
        "futures": ["atp_matches_futures"]
    }
   
    # Define year ranges for different match types
    year_ranges = {
        "singles": (1968, 2025),
        "doubles": (1968, 2025),
        "qualifiers_challengers": (1978, 2025),
        "futures": (1991, 2025)
    }
   
    # Get the appropriate prefixes and year range
    prefixes = match_types.get(data_type, [])
    start_year, end_year = year_ranges.get(data_type, (1968, 2025))
   
    # Iterate through all specified prefixes
    for prefix in prefixes:
        for year in range(start_year, end_year):
            filename = f"{prefix}_{year}.csv"
            filepath = os.path.join(base_path, filename)
           
            if os.path.exists(filepath):
                try:
                    # Expanded schema overrides to preserve string types
                    schema_overrides = {
                        'winner_seed': pl.Utf8,
                        'loser_seed': pl.Utf8,
                        'tourney_date': pl.Utf8,
                        'winner_entry': pl.Utf8,
                        'loser_entry': pl.Utf8,
                        'winner_rank': pl.Utf8,
                        'loser_rank': pl.Utf8,
                        'winner1_id': pl.Utf8,
                        'winner2_id': pl.Utf8,
                        'loser1_id': pl.Utf8,
                        'loser2_id': pl.Utf8
                    }
                   
                    df = pl.read_csv(
                        filepath,
                        schema_overrides=schema_overrides,
                        null_values=null_values,
                        ignore_errors=True,
                        truncate_ragged_lines=True,
                        infer_schema_length=10000
                    )
                   
                    # Convert tourney_date to Date type
                    df = df.with_columns(
                        pl.col('tourney_date')
                        .str.to_date(format='%Y%m%d')
                        .alias('tourney_date')
                    )
                   
                    dataframes.append(df)
                except Exception as e:
                    print(f"Error importing {filename}: {e}")
   
    # Vertical concat with relaxed schema
    return pl.concat(dataframes, how="vertical_relaxed") if dataframes else None

# Usage
singles_matches = import_tennis_matches(data_type="singles")
doubles_matches = import_tennis_matches(data_type="doubles")
qualifiers_challengers = import_tennis_matches(data_type="qualifiers_challengers")
futures_matches = import_tennis_matches(data_type="futures")

# Print shapes and date range for verification
for match_type, matches in [
    ("Singles", singles_matches),
    ("Doubles", doubles_matches),
    ("Qualifiers/Challengers", qualifiers_challengers),
    ("Futures", futures_matches)
]:
    if matches is not None:
        print(f"{match_type} matches DataFrame shape: {matches.shape}")
        print(f"{match_type} matches date range: {matches['tourney_date'].min()} to {matches['tourney_date'].max()}")

Singles matches DataFrame shape: (193337, 49)
Singles matches date range: 1967-12-28 to 2024-05-20
Doubles matches DataFrame shape: (26399, 65)
Doubles matches date range: 2000-01-03 to 2020-03-06
Qualifiers/Challengers matches DataFrame shape: (216430, 49)
Qualifiers/Challengers matches date range: 1978-01-08 to 2024-05-20
Futures matches DataFrame shape: (486553, 49)
Futures matches date range: 1990-12-29 to 2024-05-20


In [116]:
doubles_matches

tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner1_id,winner2_id,winner_seed,winner_entry,loser1_id,loser2_id,loser_seed,loser_entry,score,best_of,round,winner1_name,winner1_hand,winner1_ht,winner1_ioc,winner1_age,winner2_name,winner2_hand,winner2_ht,winner2_ioc,winner2_age,loser1_name,loser1_hand,loser1_ht,loser1_ioc,loser1_age,loser2_name,loser2_hand,loser2_ht,loser2_ioc,loser2_age,winner1_rank,winner1_rank_points,winner2_rank,winner2_rank_points,loser1_rank,loser1_rank_points,loser2_rank,loser2_rank_points,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
str,str,str,i64,str,date,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,i64,str,f64,str,str,i64,str,f64,str,str,i64,str,f64,str,str,i64,str,f64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2000-301""","""Auckland""","""Hard""",32,"""A""",2000-01-10,300,"""101704""","""101097""","""1""",null,"""101358""","""101543""","""2""",null,"""7-5 6-4""",3,"""F""","""Ellis Ferreira""","""L""",188,"""RSA""",29.889117,"""Rick Leach""","""L""",188,"""USA""",35.033539,"""Olivier Delaitre""","""R""",170,"""FRA""",32.610541,"""Jeff Tarango""","""L""",180,"""USA""",31.137577,13,0,15,0,16,0,17,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2000-301""","""Auckland""","""Hard""",32,"""A""",2000-01-10,299,"""101704""","""101097""","""1""",null,"""102562""","""101866""","""3""",null,"""6-2 6-4""",3,"""SF""","""Ellis Ferreira""","""L""",188,"""RSA""",29.889117,"""Rick Leach""","""L""",188,"""USA""",35.033539,"""Jiri Novak""","""R""",190,"""CZE""",24.804928,"""David Rikl""","""L""",178,"""CZE""",28.867899,13,0,15,0,34,0,29,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2000-301""","""Auckland""","""Hard""",32,"""A""",2000-01-10,298,"""101358""","""101543""","""2""",null,"""102664""","""101779""",null,null,"""6-3 6-4""",3,"""SF""","""Olivier Delaitre""","""R""",170,"""FRA""",32.610541,"""Jeff Tarango""","""L""",180,"""USA""",31.137577,"""Petr Pala""","""R""",193,"""CZE""",24.273785,"""Pavel Vizner""","""R""",183,"""CZE""",29.489391,16,0,17,0,97,0,74,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2000-301""","""Auckland""","""Hard""",32,"""A""",2000-01-10,297,"""101704""","""101097""","""1""",null,"""101342""","""102162""",null,"""WC""","""6-3 6-4""",3,"""QF""","""Ellis Ferreira""","""L""",188,"""RSA""",29.889117,"""Rick Leach""","""L""",188,"""USA""",35.033539,"""Patrick Galbraith""","""L""",183,"""USA""",32.736482,"""Chris Woodruff""","""R""",188,"""USA""",27.019849,13,0,15,0,65,0,248,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2000-301""","""Auckland""","""Hard""",32,"""A""",2000-01-10,296,"""101358""","""101543""","""2""",null,"""103483""","""102262""",null,"""WC""","""6-2 6-4""",3,"""QF""","""Olivier Delaitre""","""R""",170,"""FRA""",32.610541,"""Jeff Tarango""","""L""",180,"""USA""",31.137577,"""James Shortall""","""R""",null,"""NZL""",20.043806,"""Cristiano Testa""","""L""",null,"""BRA""",26.297057,16,0,17,0,779,0,114,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2020-M-DC-2020-WG2-PO-MAR-VIE-…","""Davis Cup WG2 PO: MAR vs VIE""","""Clay""",4,"""D""",2020-03-06,3,"""104917""","""104467""",null,null,"""109042""","""122533""",null,null,"""6-4 6-1""",3,"""RR""","""Anas Fattar""","""R""",null,"""MAR""",32.807666,"""Lamine Ouahab""","""R""",185,"""ALG""",35.192334,"""Quoc Khanh Le""","""R""",null,"""VIE""",37.497604,"""Nam Hoang Ly""","""R""",null,"""VIE""",23.014374,683,48,942,26,null,null,1293,12

In [117]:
doubles_matches.select(pl.col("winner_seed").value_counts()) 

winner_seed
struct[2]
"{""4"",1797}"
"{""13"",108}"
"{""15"",116}"
"{null,14787}"
"{""7"",430}"
…
"{""14"",120}"
"{""5"",531}"
"{""12"",134}"


In [118]:
singles_matches.write_parquet(file="aggregated_matches/singles_matches.parquet",compression="zstd")
doubles_matches.write_parquet(file="aggregated_matches/doubles_matches.parquet",compression="zstd")
futures_matches.write_parquet(file="aggregated_matches/futures_matches.parquet",compression="zstd")
qualifiers_challengers.write_parquet(file="aggregated_matches/quali_and_challengers_matches.parquet",compression="zstd")

In [119]:
players=pl.read_csv("matches_and_ranking/atp_players.csv")

In [ ]:
players_test=players
players_test=players_test.drop_nulls(subset="dob")
players_test.sort(by="dob",descending=False)

#peoples from 1836???

player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,name_full
i64,str,str,str,date,str,i64,str,str
113976,"""Frederick William""","""Oliver""","""U""",1836-01-04,"""GBR""",null,null,"""Frederick William Oliver"""
114158,"""Julian""","""Marshall""","""U""",1836-06-24,"""GBR""",null,"""Q1712028""","""Julian Marshall"""
114146,"""Montague""","""Hankey""","""U""",1840-08-16,"""GBR""",null,null,"""Montague Hankey"""
114004,"""Edward North""","""Buxton""","""U""",1840-09-01,"""GBR""",null,null,"""Edward North Buxton"""
113963,"""Charles Gilbert""","""Heathcote""","""U""",1841-03-02,"""GBR""",null,"""Q969559""","""Charles Gilbert Heathcote"""
…,…,…,…,…,…,…,…,…
211514,"""Justin""","""Engel""","""U""",2007-10-01,"""GER""",null,null,"""Justin Engel"""
210352,"""Santiago ""","""Lora""","""U""",2007-10-03,"""BOL""",null,null,"""Santiago Lora"""
149160,"""Jerry Christopher""","""Brow Salinas""","""U""",2007-12-03,"""PER""",null,null,"""Jerry Christopher Brow Salinas"""


In [121]:
players = players.with_columns(
    pl.concat_str(["name_first", "name_last"], separator=" ").alias("name_full")
)

players = players.with_columns(pl.col('dob').cast(pl.String, strict=False).alias('dob'))

players = players.with_columns(
                    pl.col('dob')
                    .str.to_date(format='%Y%m%d',strict=False)
                    .alias('dob')
                    )
players

player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,name_full
i64,str,str,str,date,str,i64,str,str
100001,"""Gardnar""","""Mulloy""","""R""",1913-11-22,"""USA""",185,"""Q54544""","""Gardnar Mulloy"""
100002,"""Pancho""","""Segura""","""R""",1921-06-20,"""ECU""",168,"""Q54581""","""Pancho Segura"""
100003,"""Frank""","""Sedgman""","""R""",1927-10-02,"""AUS""",180,"""Q962049""","""Frank Sedgman"""
100004,"""Giuseppe""","""Merlo""","""R""",1927-10-11,"""ITA""",null,"""Q1258752""","""Giuseppe Merlo"""
100005,"""Richard""","""Gonzalez""","""R""",1928-05-09,"""USA""",188,"""Q53554""","""Richard Gonzalez"""
…,…,…,…,…,…,…,…,…
212913,"""Pietro""","""Ricci""","""U""",null,"""ITA""",null,null,"""Pietro Ricci"""
212914,"""Corey""","""Craig""","""U""",null,"""USA""",null,null,"""Corey Craig"""
212915,"""Aleksandar""","""Ljubojevic""","""U""",null,"""SRB""",null,null,"""Aleksandar Ljubojevic"""


In [122]:
players.write_parquet(file="aggregated_matches/players.parquet",compression="zstd")

In [123]:
import polars as pl
import os

def read_atp_rankings_csv(file_path):
   schema = {
       "ranking_date": pl.Utf8,  # Keep as string initially
       "rank": pl.Int64,
       "player": pl.Utf8,
       "points": pl.Int64
   }
   
   # Read CSV with initial schema
   df = pl.read_csv(file_path, schema=schema)
   
   # Convert ranking_date to proper datetime
   df = df.with_columns(
       pl.col("ranking_date").str.strptime(pl.Date, format="%Y%m%d")
   )
   
   return df

# Read all files
periods = ['00', '10', '20', '70', '80', '90', 'current']
atp_rankings = {}

for period in periods:
   file_path = f"matches_and_ranking/atp_rankings_{period}s.csv"
   if period == 'current':
       file_path = f"matches_and_ranking/atp_rankings_current.csv"
   
   atp_rankings[f'atp_rankings{period}s'] = read_atp_rankings_csv(file_path)

In [124]:
atp_rankings

{'atp_rankings00s': shape: (920_907, 4)
 ┌──────────────┬──────┬────────┬────────┐
 │ ranking_date ┆ rank ┆ player ┆ points │
 │ ---          ┆ ---  ┆ ---    ┆ ---    │
 │ date         ┆ i64  ┆ str    ┆ i64    │
 ╞══════════════╪══════╪════════╪════════╡
 │ 2000-01-10   ┆ 1    ┆ 101736 ┆ 4135   │
 │ 2000-01-10   ┆ 2    ┆ 102338 ┆ 2915   │
 │ 2000-01-10   ┆ 3    ┆ 101948 ┆ 2419   │
 │ 2000-01-10   ┆ 4    ┆ 103017 ┆ 2184   │
 │ 2000-01-10   ┆ 5    ┆ 102856 ┆ 2169   │
 │ …            ┆ …    ┆ …      ┆ …      │
 │ 2009-12-28   ┆ 1805 ┆ 111421 ┆ 1      │
 │ 2009-12-28   ┆ 1809 ┆ 105648 ┆ 1      │
 │ 2009-12-28   ┆ 1809 ┆ 109144 ┆ 1      │
 │ 2009-12-28   ┆ 1811 ┆ 105675 ┆ 1      │
 │ 2009-12-28   ┆ 1812 ┆ 105589 ┆ 1      │
 └──────────────┴──────┴────────┴────────┘,
 'atp_rankings10s': shape: (915_618, 4)
 ┌──────────────┬──────┬────────┬────────┐
 │ ranking_date ┆ rank ┆ player ┆ points │
 │ ---          ┆ ---  ┆ ---    ┆ ---    │
 │ date         ┆ i64  ┆ str    ┆ i64    │
 ╞══════════════

In [125]:
atp_rankings_agg=atp_rankings.get('atp_rankings70s')
atp_rankings_agg.extend(atp_rankings.get('atp_rankings80s'))
atp_rankings_agg.extend(atp_rankings.get('atp_rankings90s'))
atp_rankings_agg.extend(atp_rankings.get('atp_rankings00s'))
atp_rankings_agg.extend(atp_rankings.get('atp_rankings10s'))
atp_rankings_agg.extend(atp_rankings.get('atp_rankings20s'))
atp_rankings_agg.extend(atp_rankings.get('atp_rankingscurrents'))
atp_rankings_agg     #need to have 3_235_639 rows


ranking_date,rank,player,points
date,i64,str,i64
1973-08-27,129,"""100005""",null
1973-08-27,114,"""100011""",null
1973-08-27,6,"""100016""",null
1973-08-27,19,"""100022""",null
1973-08-27,82,"""100025""",null
…,…,…,…
2024-05-27,2070,"""211388""",1
2024-05-27,2071,"""210649""",1
2024-05-27,2071,"""208890""",1


In [126]:
atp_rankings_agg.write_parquet(file='aggregated_matches/atp_rankings.parquet',compression="zstd")

In [127]:
null_values = ['Q', 'NA', 'N/A', '', '-', 'Unknown', 'null', 'W/O']
schema_overrides = {
                        'winner_seed': pl.Utf8,
                        'loser_seed': pl.Utf8,
                        'tourney_date': pl.Utf8,
                        'winner_entry': pl.Utf8,
                        'loser_entry': pl.Utf8,
                        'winner_rank': pl.Utf8,
                        'loser_rank': pl.Utf8,
                        'winner1_id': pl.Utf8,
                        'winner2_id': pl.Utf8,
                        'loser1_id': pl.Utf8,
                        'loser2_id': pl.Utf8
                    }
                   
amateur_matches = pl.read_csv(
                        source="matches_and_ranking/atp_matches_amateur.csv",
                        schema_overrides=schema_overrides,
                        null_values=null_values,
                        ignore_errors=True,
                        truncate_ragged_lines=True,
                        infer_schema_length=10000
                    )
                   
                    # Convert tourney_date to Date type
amateur_matches = amateur_matches.with_columns(
                    pl.col('tourney_date')
                    .str.to_date(format='%Y%m%d')
                    .alias('tourney_date')
                    )
amateur_matches



tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
str,str,str,str,str,date,i64,i64,str,str,str,str,str,str,f64,i64,str,str,str,str,i64,str,f64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1877-540""","""Wimbledon""","""Grass""","""32""","""G""",1877-07-09,1,113987,null,null,"""Spencer William Gore""",null,null,"""GBR""",27.331964,114009,null,null,"""Ht Gilson""",null,null,"""GBR""",null,"""6-2 6-0 6-3""",5,"""R32""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1877-540""","""Wimbledon""","""Grass""","""32""","""G""",1877-07-09,2,114146,null,null,"""Montague Hankey""",null,null,"""GBR""",36.895277,113977,null,null,"""Robert Dalby Dalby""",null,null,"""GBR""",null,"""6-4 6-2 3-6 6-2""",5,"""R32""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1877-540""","""Wimbledon""","""Grass""","""32""","""G""",1877-07-09,3,114147,null,null,"""John Baker""",null,null,"""GBR""",null,114148,null,null,"""Jw Trist""",null,null,"""GBR""",null,"""6-1 6-4 6-0""",5,"""R32""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1877-540""","""Wimbledon""","""Grass""","""32""","""G""",1877-07-09,4,114149,null,null,"""Francis Nathaniel Langham""",null,null,"""GBR""",36.082136,114150,null,null,"""Charles Francis Buller""",null,null,"""GBR""",31.12115,null,5,"""R32""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1877-540""","""Wimbledon""","""Grass""","""32""","""G""",1877-07-09,5,113953,null,null,"""Lestocq Robert Erskine""",null,null,"""GBR""",19.838467,114151,null,null,"""E Wheeler""",null,null,"""GBR""",null,"""6-2 6-5 6-2""",5,"""R32""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1967-D047""","""Davis Cup EUR R1: NED vs RSA""","""Clay""","""4""","""D""",1967-05-04,5,100084,null,null,"""Tom Okker""","""R""","""178""","""NED""",23.195072,100055,null,null,"""Cliff Drysdale""","""R""",188,"""RSA""",25.938398,"""6-2 6-2 0-6 7-5""",5,"""RR""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1967-D048""","""Davis Cup AME SF: PHI vs VIE""",null,"""4""","""D""",1967-04-14,1,106910,null,null,"""Eddie Cruz""","""R""",null,"""PHI""",null,111097,null,null,"""Van Bay Vo""","""U""",null,"""VIE""",null,"""6-2 6-3 6-2""",5,"""RR""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1967-D048""","""Davis Cup AME SF: PHI vs VIE""",null,"""4""","""D""",1967-04-14,2,112556,null,null,"""Felicisimo Ampon""",null,null,"""PHI""",46.461328,111091,null,null,"""Van Thanh Vo""","""U""",null,"""VIE""",null,"""6-4 6-1 6-1""",5,"""RR""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [128]:
atp_rankings_agg.write_parquet(file='aggregated_matches/atp_amateurs_matches.parquet',compression="zstd")